Importando bibliotecas

In [ ]:
import pandas as pd
from scipy.stats import shapiro, kstest, jarque_bera, levene

Importando base de dados categórica

In [ ]:
df = pd.read_csv("../databases/ENEM_2023_FINAL_cat.csv")
display(df)

Definindo funções de testes para verificar os requisitos de ANOVA e MANOVA

In [ ]:
def sw_p_value(df:pd.DataFrame, col:str):
    serie = df[col]
    stat, p_value = shapiro(serie)
    return p_value

def ks_p_value(df:pd.DataFrame, col:str):
    serie = df[col]
    stat, p_value = kstest(serie, 'norm', args=(serie.mean(), serie.std(ddof=1)))
    return p_value

def jb_p_value(df:pd.DataFrame, col:str):
    serie = df[col]
    stat, p_value = jarque_bera(serie)
    return p_value

def levene_p_value(subs:list[pd.Series]):
    stat, p_value = levene(*subs)
    return p_value

def anova_requests_category(sub: pd.DataFrame, cat:str, col: str):
    return {
        "Categoria": cat,
        "N": len(sub),
        "Média": sub[col].mean(),
        "Variância": sub[col].var(ddof=1),
        #"SW P-Value": sw_p_value(sub, col),
        "KS P-Value": ks_p_value(sub, col),
        "JB P-Value": jb_p_value(sub, col)
    }

def anova_requests(df:pd.DataFrame, category:str, col:str):
    groups = []
    results = []
    for cat, sub in df.groupby(category):
        groups.append(sub[col])
        results.append(anova_requests_category(sub, cat, col))
    
    return pd.DataFrame(results), levene_p_value(groups)

def all_grades_anova_requests(df:pd.DataFrame, category:str):
    grade_cols = ["NU_NOTA_CH", "NU_NOTA_CN", "NU_NOTA_MT", "NU_NOTA_LC", "NU_NOTA_REDACAO"]
    for col in grade_cols:
        results, levene_p = anova_requests(df, category, col)
        print(col)
        display(results)
        print(f"Levene P-Value: {levene_p}")
        print("-"*100)

Realizando testes de requisitos para sexo

In [ ]:
all_grades_anova_requests(df, "TP_SEXO")

Realizando testes de requisitos para cor

In [ ]:
all_grades_anova_requests(df, "TP_COR_RACA")

Realizando testes de requisitos para Escolaridade Mãe

In [ ]:
all_grades_anova_requests(df, "ESCOLARIDADE_MAE")

Realizando testes de requisitos para Escolaridade Pai

In [ ]:
all_grades_anova_requests(df, "ESCOLARIDADE_PAI")

Realizando testes de requisitos para Dependência Administrativa da Escola

In [ ]:
all_grades_anova_requests(df, "TP_DEPENDENCIA_ADM_ESC")

Relizando testes de requisitos para Níveis de Renda

In [ ]:
all_grades_anova_requests(df, "RENDA_MENSAL_CASA")

Realizando testes de requisitos para internet na casa

In [ ]:
all_grades_anova_requests(df, "INTERNET_CASA")